In [111]:
import sys
sys.path.append('../')
import time

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.signal import convolve2d
from scipy.optimize import curve_fit
from scipy.optimize import brute
# import cv2
import torch
import json
import csv
from functions.all_knots_functions import *
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
from sklearn.model_selection import train_test_split
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import collections
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import trange
import pickle
from sklearn.metrics import confusion_matrix
import seaborn as sns
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [112]:
hyperparams = {
    # 'learning_rate': 5e-4,  # Control dropout rate
    'learning_rate': 5e-4,  # Control dropout rate
    'patience': 4,  # Number of epochs between learning rate decay
    'factor': 0.2,  # Multiplicative factor of learning rate decay
    'batch_size': 64
}

num_epochs = 10
print_every = 1


In [113]:
knot_types = {
        'standard_16': 0,  # 1
        'standard_14': 1,  # 2
        'standard_18': 2,  # 3
        '30both': 3,  # 4
        '30oneZ': 4,  # 5
        'optimized': 5,  # 6
        'pm_03_z': 6,  # 7
        # '4foil': hopf_4foil,  # 8
        # '6foil': hopf_6foil,  # 9
        # 'stand4foil': hopf_stand4foil,  # 10
        '30oneX': 7,  # 11
        '15oneZ': 8,
        'dennis': 9,
        'trefoil_standard_12': 10,
        'trefoil_optimized': 11,
        # 'fivefoil_standard_08': fivefoil_standard_08

}


knots = [
    'standard_14', 'standard_16', 'standard_18', '30both', '30oneZ',
    'optimized', 'pm_03_z', '30oneX', '15oneZ', 'dennis',
    'trefoil_standard_12', 'trefoil_optimized'
]


# folders = [
#     '../HOPFS_L270_5e-07_test_1s',
#     '../HOPFS_L270_0.05_1_64x64x64_v1'
# ]
folders = [
    '../HOPFS_L270_0.05_1000_64x64x64_v1',
    '../HOPFS_L270_0.15_1000_64x64x64_v1',
    '../HOPFS_L270_0.25_1000_64x64x64_v1',
]

num_classes = len(knots)
X_list = []
Y_list = []
Xs_list = []
Ys_list = []

csv.field_size_limit(10000000)

# Loop through all folders and knots
for folder in folders:
    for knot in knots:
        filename = f'{folder}/data_{knot}_spectr.csv'
        try:
            with open(filename, 'r') as file:
                reader = csv.reader(file)
                for row in reader:
                    # Append the knot label to Ys_list
                    Ys_list.append(knot_types[knot])

                    # Load the data array from the JSON string
                    data_array = json.loads(row[0])

                    # Extract values from the data array
                    l1, l2 = data_array[0], data_array[1]
                    p1, p2 = data_array[2], data_array[3]

                    # Create and normalize the moments array
                    moments = np.array([x[0] + 1j * x[1] for x in data_array[5:]])
                    moments = moments.reshape((l2 - l1 + 1), (p2 - p1 + 1))
                    moments = moments / np.sqrt(np.sum(np.abs(moments) ** 2))

                    # Append the flattened absolute values to Xs_list
                    Xs_list.append(np.abs(moments.reshape(-1)))

        except FileNotFoundError:
            print(f'File not found: {filename}')
        except json.JSONDecodeError:
            print(f'Error decoding JSON in file: {filename}')

print(f'Loaded {len(X_list)} samples.  {int(len(X_list) / len(knots))} per class')

File not found: ../HOPFS_L270_0.05_1000_64x64x64_v1/data_standard_14_spectr.csv
File not found: ../HOPFS_L270_0.05_1000_64x64x64_v1/data_standard_16_spectr.csv
File not found: ../HOPFS_L270_0.05_1000_64x64x64_v1/data_standard_18_spectr.csv
File not found: ../HOPFS_L270_0.05_1000_64x64x64_v1/data_30both_spectr.csv
File not found: ../HOPFS_L270_0.05_1000_64x64x64_v1/data_30oneZ_spectr.csv
File not found: ../HOPFS_L270_0.05_1000_64x64x64_v1/data_optimized_spectr.csv
File not found: ../HOPFS_L270_0.05_1000_64x64x64_v1/data_pm_03_z_spectr.csv
File not found: ../HOPFS_L270_0.05_1000_64x64x64_v1/data_30oneX_spectr.csv
File not found: ../HOPFS_L270_0.05_1000_64x64x64_v1/data_15oneZ_spectr.csv
File not found: ../HOPFS_L270_0.05_1000_64x64x64_v1/data_dennis_spectr.csv
File not found: ../HOPFS_L270_0.05_1000_64x64x64_v1/data_trefoil_standard_12_spectr.csv
File not found: ../HOPFS_L270_0.05_1000_64x64x64_v1/data_trefoil_optimized_spectr.csv
File not found: ../HOPFS_L270_0.15_1000_64x64x64_v1/data_

In [114]:
Xs_np = np.array(Xs_list)
ys_np = np.array(Ys_list)
# print(ys_np.shape)
Xs_torch = torch.tensor(Xs_np).float()
ys_t = torch.tensor(ys_np)
ys_torch = F.one_hot(ys_t.long(), num_classes=num_classes).float()
print(Xs_torch.shape, ys_torch.shape)

torch.Size([0]) torch.Size([0, 12])


In [115]:
print(Xs_torch[0])

IndexError: index 0 is out of bounds for dimension 0 with size 0

In [ ]:
X_train, X_val_test, y_train, y_val_test = train_test_split(Xs_torch, ys_torch, test_size=0.3, random_state=37)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=37)
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

In [ ]:


# Helper function to create a convolutional stage
def conv_stage_2d(layer_configs):
    layers = []
    for config in layer_configs:
        in_channels, out_channels, kernel_size, stride, padding = config
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding))
        layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.ReLU(inplace=True))
    return nn.Sequential(*layers)

# Helper function to create a pooling layer
def create_pooling_layer_2d(config):
    if config is None:
        return None
    kernel_size, stride, padding = config
    return nn.MaxPool2d(kernel_size=kernel_size, stride=stride, padding=padding)

# 2D Convolutional Classifier
class Classifier2D(nn.Module):
    def __init__(self, stages, pooling_configs, num_classes=11, shape_X_l=7, shape_X_p=13):
        super(Classifier2D, self).__init__()

        self.shape_X_l = shape_X_l
        self.shape_X_p = shape_X_p

        self.features = nn.Sequential()

        # Add convolutional stages and pooling layers
        for i, stage in enumerate(stages):
            self.features.add_module(f"stage_{i}", conv_stage_2d(stage))

            # Add MaxPooling layer after each stage based on the pooling configuration
            if i < len(pooling_configs):
                pool_layer = create_pooling_layer_2d(pooling_configs[i])
                if pool_layer:
                    self.features.add_module(f"pool_{i}", pool_layer)

        # Calculate the size of the flattened features after the conv layers
        self._to_linear = None
        self._get_conv_output((1, shape_X_l, shape_X_p))  # Add a channel dimension here

        # Fully connected layers
        self.fc1 = nn.Linear(self._to_linear, 256)
        self.fc2 = nn.Linear(256, num_classes)

    # Helper function to calculate the output size after convolution and pooling
    def _get_conv_output(self, shape):
        batch_size = 1
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))
        output_feat = self.features(input)
        self._to_linear = int(np.prod(output_feat.size()[1:]))

    # Optional: Initialize weights with specific methods
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    # Forward pass
    def forward(self, x):
        # Reshape input from (batch, shape_X_l * shape_X_p) -> (batch, 1, shape_X_l, shape_X_p)
        batch_size = x.size(0)
        x = x.view(batch_size, 1, self.shape_X_l, self.shape_X_p)  # Add a channel dimension

        # Pass through convolutional layers
        x = self.features(x)

        # Flatten the output
        x = x.view(x.size(0), -1)

        # Pass through fully connected layers
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x




In [ ]:

stages = [
    # Stage 1: (in_channels, out_channels, kernel_size, stride, padding)
    [(1, 16, 3, 1, 1)],  # First convolution stage with padding
    [(16, 32, 3, 1, 1)],  # Second convolution stage with padding
    [(32, 64, 3, 1, 1)]   # Third convolution stage with padding
]

# Define pooling configurations for each stage (kernel_size, stride, padding)
pooling_configs = [
    (2, 2, 0),  # MaxPool after first stage
    None,        # Skip pooling after the second stage
    (2, 2, 0)   # MaxPool after the third stage
]
stages = [
    [(1, 32, 3, 1, 1), (32, 32, 3, 1, 1), (32, 32, 3, 1, 1)],
    [(32, 64, 5, 1, 1), (64, 64, 5, 1, 1), (64, 64, 5, 1, 1)]
]
stages = [
    [(1, 32, 3, 1, 1), (32, 32, 3, 1, 1), (32, 32, 3, 1, 1), (32, 32, 3, 1, 1)],
    [(32, 64, 3, 1, 1), (64, 64, 3, 1, 1), (64, 64, 3, 1, 1), (64, 64, 3, 1, 1)]
]
# Define pooling configurations: (kernel_size, stride, padding)
# Set to 'None' for stages where no pooling is desired
# kernel_size, stride, padding
pooling_configs = [
    (2, 2, 1),  # Pooling after the first stage
    (2, 2, 1)      # No pooling after the third stage
]
# Example usage
shape_X_l = 7  # Modify as needed
shape_X_p = 13  # Modify as needed
num_classes = ys_torch.shape[-1]

# Initialize the model
model = Classifier2D(stages, pooling_configs, num_classes, shape_X_l, shape_X_p).to(device)

print(model)

In [ ]:
# print(model._to_linear, 512 * 16 * 16)
dots_3d_toch_batch = train_dataset[1:4][0].to(device)
print(dots_3d_toch_batch.shape)
print(model(dots_3d_toch_batch).shape)
summary(model, input_size=dots_3d_toch_batch.shape[1:])

In [ ]:
train_dataset[1:2][0]

In [ ]:
def loop_train(model, train_loader, criterion, optimizer):
    model.train()  # Set the model to training mode
    total_loss = 0
    for i, (inputs, targets) in enumerate(train_loader, 1):  # Start enumeration from 1
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()  # Clear the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights
        total_loss += loss.item()  # Accumulate the loss

    return total_loss / len(train_loader)  # Return the average loss


def loop_test(model, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    with torch.no_grad():  # No need to track the gradients
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, targets)  # Compute the loss
            total_loss += loss.item()  # Accumulate the loss
    return total_loss / len(test_loader)  # Return the average loss


def correct_number_test(model, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    total_correct = 0
    total = 0
    with torch.no_grad():  # No need to track the gradients
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)  # Forward pass
            correct = criterion(outputs, targets)  # Compute correct
            total_correct += correct  # Accumulate correct
            total += len(outputs)  # Accumulate total
    return total_correct, total  # Return the average loss


def plot_losses(train_losses, test_losses, decimals=3):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses,
             label=f'Train Loss {train_losses[-1]: .{decimals}f} (min: {min(train_losses): .{decimals}f})')
    plt.plot(test_losses, label=f'Test Loss {test_losses[-1]: .{decimals}f} (min: {min(test_losses): .{decimals}f})')
    plt.title('Training and Testing Losses Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


In [ ]:

# criterion = nn.MSELoss().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams['learning_rate'])
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=hyperparams['factor'], patience=hyperparams['patience'],
                              verbose=True)



train_losses = []
val_losses = []
train_loader = DataLoader(train_dataset, batch_size=hyperparams['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=hyperparams['batch_size'])
test_loader = DataLoader(test_dataset, batch_size=hyperparams['batch_size'])
start_time = time.time()
for epoch in trange(num_epochs, desc="Progress"):
    epoch_start_time = time.time()
    train_losses.append(loop_train(model, train_loader, criterion, optimizer))
    val_losses.append(loop_test(model, val_loader, criterion))

    scheduler.step(val_losses[-1])

    if (epoch + 1) % print_every == 0:
        epoch_time = time.time() - epoch_start_time
        print(f'Epoch {epoch} completed in {epoch_time:.2f} seconds')
        print(f'Epoch {epoch}: Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}')


total_training_time = time.time() - start_time
print(f'Total training time: {total_training_time:.2f} seconds')

In [ ]:
plot_losses(train_losses, val_losses, decimals=3)

In [ ]:
# predictions = model(X_torch.to(device))
# predictions
model.eval()  # Set the model to evaluation mode
predicted_labels = []
with torch.no_grad():  # Disable gradient calculation
    for batch in test_loader:
        inputs = batch[0].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predicted_labels.extend(predicted.cpu().numpy())
# Convert the list to a numpy array
predicted_labels = np.array(predicted_labels)
# If you need the result as a tensor, convert back to tensor
predicted_labels = torch.tensor(predicted_labels)

# _, predicted_labels = torch.max(model(X_test.to(device)), 1)
_, true_class_labels = torch.max(y_test, 1)
predicted_labels_np = predicted_labels.cpu().numpy()
true_labels_np = true_class_labels.cpu().numpy()
cm = confusion_matrix(true_labels_np, predicted_labels_np)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=knots, yticklabels=knots)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
Accuracy = (np.sum(predicted_labels_np == true_labels_np)) / len(predicted_labels_np)
print(Accuracy)

In [ ]:
# Save the model state (weights) and structure
model_save_path = "classifier2d.pth"

torch.save({
    'model_state_dict': model.state_dict(),
    'hyperparams': hyperparams,
    'num_classes': num_classes,
    'stages': stages,
    'pooling_configs': pooling_configs,
}, model_save_path)

print(f"Model saved to {model_save_path}")